![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_GM_DE.ipynb)

###  Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [4]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.4.4
Spark NLP_JSL Version : 4.4.4


# **Models**

# `sbertresolve_icd10gm`

This model maps clinical findings to their corresponding ICD-10-GM code in healthcare records using Entity Resolvers.

### Construct the pipeline

**Create the pipeline**

In [5]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
    .setInputCols(["sentence", 'token'])\
    .setOutputCol("embeddings")

ner = MedicalNerModel.pretrained('ner_healthcare','de', 'clinical/models') \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\

c2doc = Chunk2Doc()\
    .setInputCols("ner_chunk")\
    .setOutputCol("ner_chunk_doc")

sbert_embedder = BertSentenceEmbeddings.pretrained("sent_bert_base_cased", "de")\
    .setInputCols(["ner_chunk_doc"])\
    .setOutputCol("sbert_embeddings")

icd10gm_resolver = SentenceEntityResolverModel.pretrained("sbertresolve_icd10gm", "de", "clinical/models") \
    .setInputCols(["sbert_embeddings"]) \
    .setOutputCol("icd10gm_code")


nlp_pipeline = Pipeline(
    stages = [
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10gm_resolver
        ])


w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
[OK!]
sent_bert_base_cased download started this may take some time.
Approximate size to download 390.2 MB
[OK!]
sbertresolve_icd10gm download started this may take some time.
[OK!]


###  Create example inputs

In [6]:
sample_text = [
"""Hallo. Ich habe folgendes Problem. Ich leide seit Jahren unter chronischen polypen. Ich habe sie mir bisher schon einmal operativ entfernen lassen, nach Monaten waren sie wieder da. Da ich seit gut einem Jahr auch nichts mehr riechen und schmecken kann, suche ich mach einer Lösung. Jetzt war ich beim Spezialisten und habe mir sagen lassen ,dass es eine neue Methode und zwar mit der Antikörper Therapie sollen die polypen unterdrückt werden bzw. Sie sollen sich zurück bilden. Hatte da jemand Erfahrungen mit gemacht? Wie läuft das genau ab? Muss ich mir dann mein Leben lang antikörper spritzen lassen? Ich war beim Arzt zu aufgeregt um alles zu fragen.
@katharina. finde es einfadh nur heftig das der arzt gleich mit einer 100 er dosis anfängt , man sollte doch langsam steigern, und das allmälich.. ich selbst hatte anfangs einen tsh von 3,75 jetzt bin ich bei 0,87 also voll in der norm. und ich nehm gerade mal 50µ.... vllt solltestd u ml zu einem anderen artz gehen???
ich will dir nur aufzeigen, dass gegenseitiges verständnis auch ein stück weit dazu führen kann, dass die ganze beziehung zwischen euch drei etwas weniger anstrengend für ihn wird. und ja, das ist doof, weil er kein verständnis für euch aufbringt, ihr aber für ihn. aber so ist das leben mit einem autisten nunmal - er hat sich seinen autismus ebenso wenig ausgesucht, wie ihr.""",

"""Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ich seit dem nicht mehr losgeworden bin. Vom Symbioflor2 bekam ich dann auch immer stärkere körperliche Beschwerden, so daß ich es mit der Angst zu tun bekam und meinen Arzt bat mir Cotrim zuverschreiben (das zweitemal nach 1983). Die Folge war leider, daß ich starken Fußschweiß bekam mit entsprechendem Geruch. Auch meine Hände fingen stark an zu riechen.
Ich war im Schlaflabor. Nur eine Auffälligkeit. In der ersten Nacht dort trat ich verfrüht in den Traumschlaf ein, was ein Kriterium für Narkolepsie ist. Bluttest und Gentest aber waren negativ. Meine Lungen sind super, mein MRT unauffällig. Jetzt bin ich in einer neurologischen Spezialklinik. Der Arzt meinte, dass eine sehr geringe Wahrscheinlichkeit besteht, dass ich doch an Narkolepsie leide. Ich soll in einem halben Jahr nochmal dort ins Schlaflabor. Medikamente gegen die Müdigkeit will er mir nicht verschreiben. Antidepressiva wären es, aber die müsste ich mir abgewöhnen, bevor ich in das Schlaflabor gehe. Also lieber "clean" bleiben.
Ich glaube mittlerweile, dass das Problem ganz woanders liegt... obwohl ich als Kind laut Aussagen meiner Eltern immer recht schnell dabei war, was laufen, Fahrradfahren etc. angeht, habe ich jetzt gerade als Erwachsener gemerkt, dass ich sehr langsam dabei bin, komplexe Bewegungen einzulernen und richtig auszuführen, sei es z.B. Tanzen oder Kampfsport. Wobei das sicherlich was ist, was mit der Zeit und Training schon deutlich besser geworden ist.""",

"""Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich schon sehr wohl da gefühlt. Haben das auch von den Betreuern als Rückmeldung erhalten. Er ist zwar immer furchtbar aufgeregt wenn es dann los geht, aber wenn er dann da ist, freut er sich doch. Müssen zwar dann auch meist er sehr lange Autofahrt in Kauf nehmen da es das nicht in unserer Gegend gibt. Aber wenn er wenigstens daran eine Freude, warum nicht.
Leute, lest euch diesen Beitrag vollständig durch! Cosma und ich, wir haben unsere Erlebnisse gut dokumentiert. Wir waren am Anfang genauso ratlos, wie du (und noch andere, die hier dieselben Fragen stellem), die Diagnose ist für solche Sachen entweder Depressionen oder Burnout bzw. beides (wenn nichts Körperliches vorliegt). Manche haben noch zusätzlich Angstzustände oder Panikattacken. Die Diagnose sollte allerdings ein Neurologe/Psychiater stellen.""",

"""Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer Länder, u. a. GB. Also ich bin zum Besispiel einer von den süßen Opis, die derzeit bloß nicht an den Eiern gedrückt werden sollen. Ich würde mir tatsächlich eine Ansteckung mit einem einigermaßen moderaten Verlauf wünschen. Dann hätte ich, nach allem was man weiß, u. a. von Rhesusaffen, eine Grundimmunität und könnte rausgehen und meinen Mitmenschen unter die Arme greifen. Ich bin halt alt, aber net unbedingt schwach. Rüstiger Greis, sozusagen"""]


In [7]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text,StringType()).toDF('text')

df.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|Hallo. Ich habe folgendes Problem. Ich leide seit Jahren unter chronischen polypen. Ich habe sie ...|
|Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ic...|
|Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich sch...|
|Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer L...|
+----------------------------------------------------------------------------------------------------+



###  Use the pipeline to create outputs

In [8]:
# returns spark df resolution results

def get_codes_from_df(result_df, chunk, output_col, hcc= False):


    if hcc:

        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result,
                                                     result_df[chunk].metadata,
                                                     result_df[output_col].result,
                                                     result_df[output_col].metadata)).alias("cols")) \
                      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                              F.expr("cols['0']").alias("ner_chunk"),
                              F.expr("cols['1']['entity']").alias("entity"),
                              F.expr("cols['2']").alias("icd10_code"),
                              F.expr("cols['3']['all_k_results']").alias("all_codes"),
                              F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                              F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()



        codes = []
        resolutions = []
        hcc_all = []

        for code, resolution, hcc in zip(df['all_codes'], df['resolutions'], df['hcc_list']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))
            hcc_all.append(hcc.split(":::"))

        df['all_codes'] = codes
        df['resolutions'] = resolutions
        df['hcc_list'] = hcc_all

    else:

        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result,
                                                     result_df[chunk].metadata,
                                                     result_df[output_col].result,
                                                     result_df[output_col].metadata)).alias("cols")) \
                      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                              F.expr("cols['0']").alias("ner_chunk"),
                              F.expr("cols['1']['entity']").alias("entity"),
                              F.expr("cols['2']").alias(f"{output_col}"),
                              F.expr("cols['3']['all_k_results']").alias("all_codes"),
                              F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



        codes = []
        resolutions = []

        for code, resolution in zip(df['all_codes'], df['resolutions']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))

        df['all_codes'] = codes
        df['resolutions'] = resolutions


    return df

In [9]:
result = nlp_pipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.icd10gm_code.result,
                                     result.icd10gm_code.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("entity"),
              F.expr("cols['4']").alias("icd10_code"),
              F.expr("cols['5']['all_k_results']").alias("all_codes"),
              F.expr("cols['5']['all_k_resolutions']").alias("resolutions")).show(truncate=30)

+-------------------+-----+---+--------------------+----------+------------------------------+------------------------------+
|              chunk|begin|end|              entity|icd10_code|                     all_codes|                   resolutions|
+-------------------+-----+---+--------------------+----------+------------------------------+------------------------------+
|        seit Jahren|   45| 55|    TIME_INFORMATION|     M83.8|M83.8:::T89.02:::S92.28:::M...|Sonstige Osteomalazie im Er...|
|chronischen polypen|   63| 81|   MEDICAL_CONDITION|       I83|I83:::Z94.7:::Z94.5:::Z00.3...|Varizen der unteren Extremi...|
|           operativ|  121|128|      MEDICAL_DEVICE|     B35.8|B35.8:::I46:::I50:::Z30:::Z...|Sonstige Dermatophytosen:::...|
|          entfernen|  130|138|           TREATMENT|    R50.80|R50.80:::R09.2:::I46.0:::O8...|Fieber unbekannter Ursache:...|
|       nach Monaten|  148|159|    TIME_INFORMATION|     M83.8|M83.8:::P37.4:::M80.8:::T90...|Sonstige Osteomalazie im

###  Visualize results

In [10]:
from sparknlp_display import EntityResolverVisualizer

resolver_viz = EntityResolverVisualizer()


for j in range(len(sample_text)):
    resolver_viz.display(result = result.collect()[j], label_col = "ner_chunk", resolution_col="icd10gm_code")
    print("\n\n")

# `robertaresolve_icd10gm`

### Construct the first pipeline

In [5]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")\

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d",'de','clinical/models')\
  .setInputCols(["sentence", 'token'])\
  .setOutputCol("embeddings")

clinical_ner_slim = MedicalNerModel.pretrained("ner_healthcare_slim",'de','clinical/models') \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("slim_ner")\

clinical_ner_converter_slim = NerConverterInternal() \
  .setInputCols(["sentence", "token", "slim_ner"]) \
  .setOutputCol("slim_ner_chunk")\
  .setWhiteList(['MEDICAL_CONDITION'])

chunkerFilter = ChunkFilterer() \
  .setInputCols(["document","slim_ner_chunk"]) \
  .setOutputCol("filtered") \
  .setCriteria("isin") \
  .setBlackList(['Prävalenz',"betroffen","Risikofaktoren","Exposition","Todesursache","Krebsart","Karzinomen","sklerosierenden","Karzinomkomponenten","Invasion","Metastasierung","Metastasen","Krebsarten","dMMR-Lungenkrebs" "Neoantigene", "Strukturen","Kindesalter","diagnostiziert","Malignität","Subtypen","klarzelliges","SCLC","Heiserkeit","Obstruktion","Risiko","Vena-cava-Syndrom","Strahlenexposition","Metastasen","muzinöses","gemischtzelliges","UC-Komponente","1/2","Adenokarzinom","Fernmetastasierung","dMMR","dMMR-Lungenkrebs"])

clinical_ner_pipeline = Pipeline(
      stages = [
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner_slim,
      clinical_ner_converter_slim,
      chunkerFilter,
      ])

empty_df = spark.createDataFrame([['']]).toDF("text")

clinical_ner_model = clinical_ner_pipeline.fit(empty_df)



w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare_slim download started this may take some time.
[OK!]


###  Create example inputs

In [6]:
text_list = [
"""Das Kleinzellige Bronchialkarzinom (Kleinzelliger Lungenkrebs, SCLC) ist Hernia femoralis, Akne, einseitig, ein hochmalignes bronchogenes Karzinom, das überwiegend im Zentrum der Lunge, in einem Hauptbronchus entsteht. Die mittlere Prävalenz wird auf 1/20.000 geschätzt. Vom SCLC sind hauptsächlich Peronen mittleren Alters (27-66 Jahre) mit Raucheranamnese betroffen. Etwa 70% der Patienten mit SCLC haben bei Stellung der Diagnose schon extra-thorakale Symptome. Zu den Symptomen gehören Thoraxschmerz, Dyspnoe, Husten und pfeifende Atmung. Die Beteiligung benachbarter Bereiche verursacht Heiserkeit, Dysphagie und Oberes Vena-cava-Syndrom (Obstruktion des Blutflusses durch die Vena cava superior). Zusätzliche Symptome als Folge einer Fernmetastasierung sind ebenfalls möglich. Rauchen und Strahlenexposition sind synergistisch wirkende Risikofaktoren. Die industrielle Exposition mit Bis (Chlormethyläther) ist ein weiterer Risikofaktor. Röntgenaufnahmen des Thorax sind nicht ausreichend empfindlich, um einen SCLC frühzeitig zu erkennen. Röntgenologischen Auffälligkeiten muß weiter nachgegangen werden, meist mit Computertomographie. Die Diagnose wird bioptisch gesichert. Patienten mit SCLC erhalten meist Bestrahlung und/oder Chemotherapie. In Hinblick auf eine Verbesserung der Überlebenschancen der Patienten ist sowohl bei ausgedehnten und bei begrenzten SCLC eine kombinierte Chemotherapie wirksamer als die Behandlung mit Einzelsubstanzen. Es kann auch eine prophylaktische Bestrahlung des Schädels erwogen werden, da innerhalb von 2-3 Jahren nach Behandlungsbeginn ein hohes Risiko für zentralnervöse Metastasen besteht. Das Kleinzellige Bronchialkarzinom ist der aggressivste Lungentumor: Die 5-Jahres-Überlebensrate beträgt 1-5%, der Median des gesamten Überlebens liegt bei etwa 6 bis 10 Monaten.""",
"""Das kleinzellige Blasenkarzinom ist eine Art seltener und hochaggressiver Tumor, der in einem fortgeschrittenen Stadium vorliegt und zur frühen Metastasierung neigt.Als Behandlung wurden Chirurgie, Chemotherapie und Strahlentherapie entweder allein oder als „Teil einer kombinierten Therapie“ eingesetzt.Alle Tumorgewebe enthielten kleinzellige Karzinomkomponenten. 4 Fälle koexistierten mit anderen histologischen Arten von Blasenkrebs und 2 von 9 Fällen hatten drei verschiedene Zellkomponenten. Alle Patienten hatten eine Muskelinvasion, und 4 Fälle zeigten Lymphknotenmetastasen, 3 Fälle zeigten eine Invasion benachbarter Strukturen (Samenbläschen oder Uterus) und 1 Fall wurde mit hohem Verdacht auf Lebermetastasen.Die häufigste Todesursache war ein Tumorrezidiv oder eine Metastasierung, während 2 Patienten noch am Leben waren.Kleinzellige Blasenkarzinom unterscheidet sich vom Übergangszellkarzinom der Blase. Da die Krankheit leicht frühe Metastasen entwickelt, ist die Gesamtprognose dieser Krebsart schlecht."""
"""Mismatch-Reparatur-defiziente (dMMR) Krebsarten erzeugen eine beträchtliche Anzahl immunogener Neoantigene, was sie empfindlich für eine Immuntherapie macht. Dennoch gibt es eine beträchtliche Variabilität beim Ansprechen, und ungefähr die Hälfte der dMMR-Krebserkrankungen sind refraktär gegenüber einer Immuntherapie. Hier untersuchen wir einen Patienten mit dMMR-Lungenkrebs, der gegenüber einer Immuntherapie refraktär ist. Bei der Vorstellung in unserer Klinik wurde festgestellt, dass sie im Alter von 48 Jahren eine persönliche Vorgeschichte von Endometriumkarzinom hatte und eine Familienanamnese von früh einsetzendem Dickdarmkrebs und Endometriumkarzinom """,
"""Dedifferenziertes endometrioides Adenokarzinom ist selten und bekanntermaßen aggressiver als hochgradiges endometrioides Karzinom. Dies ist eine retrospektive Studie mit vier Fällen von Dedifferenziertes endometrioides Adenokarzinom der Gebärmutter, die zwischen Januar 2013 und Dezember 2015 in unserer Abteilung für Geburtshilfe und Gynäkologie diagnostiziert und behandelt wurden. Jeder Tumor bestand aus einem undifferenzierten Karzinom und einem niedriggradigen endometrioiden Karzinom mit abruptem Übergang zwischen ihnen. Dedifferenziertes endometrioides Adenokarzinom sollte aufgrund seines fulminanten klinischen Verlaufs nicht als konventionelles endometrioides Adenokarzinom unterdiagnostiziert werden. Das Endometriumkarzinom ist die häufigste gynäkologische Malignität in den Industrieländern. Es umfasst mehrere pathologische Subtypen, wie endometrioides, muzinöses, klarzelliges, gemischtzelliges, undifferenziertes und dedifferenziertes Karzinom. Das undifferenzierte Karzinom des Endometriums wird als bösartige Neubildung ohne Differenzierung definiert. Es zeigt ein solides, musterloses Wachstum und hat ein schlechteres klinisches Ergebnis als ein hochgradiges endometrioides Adenokarzinom. Es wird oft in Kombination mit einem differenzierten endometrioiden Karzinom (Grad 1/2) beobachtet, das als dedifferenziertes endometrioides Adenokarzinom bezeichnet wird. Es ist bekannt, dass die biologischen Merkmale von Dedifferenziertes endometrioides Adenokarzinom durch die UC-Komponente bestimmt werden, selbst wenn diese Komponente 20  der gesamten Neoplasien ausmacht, also mit aggressivem Ausgang. """
]

In [7]:
files = [f"{i}.txt" for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'text': text_list, 'file' : files}))

###  Use the pipeline to create outputs

In [8]:
result = clinical_ner_model.transform(df)

In [9]:
result_pd = result.select('file', F.explode(F.arrays_zip(result.filtered.result,result.filtered.metadata,result.filtered.begin,result.filtered.end)).alias("cols")) \
                  .select('file', F.expr("cols['0']").alias("chunk"),
                                  F.expr("cols['1']['entity']").alias("entity"),
                                  F.expr("cols['2']").alias("begin"),
                                  F.expr("cols['3']").alias("end")) \
                  .toPandas()

In [10]:
result_pd

,file,chunk,entity,begin,end
0,1.txt,Bronchialkarzinom,MEDICAL_CONDITION,17,33
1,1.txt,Lungenkrebs,MEDICAL_CONDITION,50,60
2,1.txt,Hernia,MEDICAL_CONDITION,73,78
3,1.txt,Akne,MEDICAL_CONDITION,91,94
4,1.txt,hochmalignes bronchogenes Karzinom,MEDICAL_CONDITION,112,145
5,1.txt,Symptome,MEDICAL_CONDITION,455,462
6,1.txt,Symptomen,MEDICAL_CONDITION,472,480
7,1.txt,Thoraxschmerz,MEDICAL_CONDITION,490,502
8,1.txt,Dyspnoe,MEDICAL_CONDITION,505,511
9,1.txt,Husten,MEDICAL_CONDITION,514,519


In [11]:
result_pd = result_pd.rename(columns={'chunk': 'text'})

### Construct the second pipeline

In [12]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

roberta_embeddings = XlmRoBertaEmbeddings.pretrained("xlmroberta_embeddings_paraphrase_mpnet_base_v2", "xx")\
    .setInputCols(["document",'token'])\
    .setOutputCol("word_embeddings")\
    .setCaseSensitive(True)

sentence_embeddings = SentenceEmbeddings()\
    .setInputCols("document", "word_embeddings")\
    .setOutputCol("sentence_embeddings")

icd10gm_resolver = SentenceEntityResolverModel.pretrained("robertaresolve_icd10gm", "de", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("icd10gm_code")

icd10gm_pipeline = Pipeline(
    stages = [
        documentAssembler,
        tokenizer,
        roberta_embeddings,
        sentence_embeddings,
        icd10gm_resolver])

icd10gm_model = icd10gm_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

xlmroberta_embeddings_paraphrase_mpnet_base_v2 download started this may take some time.
Approximate size to download 974.6 MB
[OK!]
robertaresolve_icd10gm download started this may take some time.
[OK!]


In [13]:
spark_df = spark.createDataFrame(result_pd)

result = icd10gm_model.transform(spark_df)

In [14]:
result_pd_last= result.select('file', F.explode(F.arrays_zip(result.document.result,result.icd10gm_code.result, result.icd10gm_code.metadata,)).alias("cols"))\
        .select('file', F.expr("cols['0']").alias("text"),
                        F.expr("cols['1']").alias("icd10gm_code"),
                        F.expr("cols['2']['resolved_text']").alias("icd10gm_description"),
                        F.expr("cols['2']['all_k_results']").alias("all_codes"),
                        F.expr("cols['2']['all_k_resolutions']").alias("resolutions"),
                        ).toPandas()

In [15]:
result_pd_last

,file,text,icd10gm_code,icd10gm_description,all_codes,resolutions
0,1.txt,Bronchialkarzinom,J47,Bronchiektasen,J47:::J20:::J21:::J41.1:::Q33.4:::Q32.2:::Q32.3:::J41.0:::J45:::Z96.80:::J20.8:::J20.6:::J41:::D02.2:::J41.8:::J21.8:::J20.3:::J42:::J20.9:::J20.2:::D14.3:::J21.9:::J21.0:::T17.5:::J45.1,Bronchiektasen:::Akute Bronchitis:::Akute Bronchiolitis:::Schleimig-eitrige chronische Bronchitis:::Angeborene Bronchiektasie:::Angeborene Bronchomalazie:::Angeborene Bronchusstenose:::Einfache ch...
1,1.txt,Lungenkrebs,J81,Lungenödem,J81:::A20.2:::A22.1:::I26:::Q22.0:::Z52.80:::B45.0:::I37.1:::Q33.3:::I37:::B40.1:::J98.1:::I37.0:::I37.8:::B46.0:::S27.32:::Q33.1:::C34:::B40.0:::C78.0:::Q26.3:::J44:::A42.0:::D02.2:::D14.3,Lungenödem:::Lungenpest:::Lungenmilzbrand:::Lungenembolie:::Pulmonalklappenatresie:::Lungenspender:::Kryptokokkose der Lunge:::Pulmonalklappeninsuffizienz:::Agenesie der Lunge:::Pulmonalklappenkra...
2,1.txt,Hernia,K42,Hernia umbilicalis,K42:::K40:::K41:::K43:::K44:::O26.4:::K45:::I25.3:::K42.1:::B00.7:::K43.78:::L13.0:::B00.2:::Q99.1:::K44.1:::Q56.0:::K43.4:::K40.1:::K41.1:::B00.78:::K46:::B00.8:::K43.79:::B00.70:::K43.7,Hernia umbilicalis:::Hernia inguinalis:::Hernia femoralis:::Hernia ventralis:::Hernia diaphragmatica:::Herpes gestationis:::Sonstige abdominale Hernien:::Herz-(Wand-)Aneurysma:::Hernia umbilicalis...
3,1.txt,Akne,L70,Akne,L70:::L70.8:::L70.5:::L70.9:::L98.5:::A22.0:::H31:::L02.0:::C44:::D44.0:::M24.1:::O90.2:::I86.80:::M85.4:::A20.1:::T86.51:::C79.2:::C44.9:::L13:::C44.59:::H15:::M50:::L44:::H04.1:::Z52.3,"Akne:::Sonstige Akne:::Acné excoriée:::Akne, nicht näher bezeichnet:::Muzinose der Haut:::Hautmilzbrand:::Sonstige Affektionen der Aderhaut:::Hautabszess, Furunkel und Karbunkel im Gesicht:::Sonst..."
4,1.txt,hochmalignes bronchogenes Karzinom,T17.5,Fremdkörper im Bronchus,T17.5:::Z96.80:::J41.1:::Q33.1:::Q32.3:::D14.3:::J20.8:::Q24.3:::S27.31:::Q33.4:::J21.8:::R06.5:::A22.1:::J81:::Q32.4:::J21:::J20.9:::I42.8:::J39.1:::K83.5:::Q24.1:::C39.9:::M31.6:::J20:::J21.9,Fremdkörper im Bronchus:::Vorhandensein eines Bronchialstents:::Schleimig-eitrige chronische Bronchitis:::Akzessorischer Lungenlappen:::Angeborene Bronchusstenose:::Gutartige Neubildung: Bronchus ...
5,1.txt,Symptome,R45,"Symptome, die die Stimmung betreffen",R45:::R39:::R45.8:::R39.8:::R46:::F68.0:::F34:::R46.8:::R29:::R68.1:::F45.4:::R63:::F34.9:::R63.8:::R29.8:::R44:::R44.8:::N64.5:::R68.8:::R09:::R09.8:::F32.3:::M05.08:::F38.8:::U63.1,"Symptome, die die Stimmung betreffen:::Sonstige Symptome, die das Harnsystem betreffen:::Sonstige Symptome, die die Stimmung betreffen:::Sonstige und nicht näher bezeichnete Symptome, die das Harn..."
6,1.txt,Symptomen,R39,"Sonstige Symptome, die das Harnsystem betreffen",R39:::R45:::R45.8:::R39.8:::R46:::F34:::F68.0:::R46.8:::R29:::R68.1:::R63:::F34.9:::F45.4:::R63.8:::R29.8:::R44:::R44.8:::R09:::N64.5:::R68.8:::R09.8:::F32.3:::F38.8:::M05.08:::U63.1,"Sonstige Symptome, die das Harnsystem betreffen:::Symptome, die die Stimmung betreffen:::Sonstige Symptome, die die Stimmung betreffen:::Sonstige und nicht näher bezeichnete Symptome, die das Harn..."
7,1.txt,Thoraxschmerz,C76.1,Thorax,C76.1:::M96.81:::T33.2:::S20.2:::M41.35:::M41.34:::S22.5:::S20:::S29.8:::T34.2:::Z93.80:::M48.45:::S20.7:::S29:::Z43.80:::T04.1:::S21:::M41.3:::S20.8:::S29.7:::M49.54:::Q67.8:::S20.88:::S29.9:::M4...,Thorax:::Instabiler Thorax nach thoraxchirurgischem Eingriff:::Oberflächliche Erfrierung des Thorax:::Prellung des Thorax:::Thoraxbedingte Skoliose: Thorakolumbalbereich:::Thoraxbedingte Skoliose:...
8,1.txt,Dyspnoe,R06.0,Dyspnoe,R06.0:::R13:::R30.0:::N94.1:::F34.1:::G24:::R49.0:::G24.0:::J98.10:::Q77.7:::G24.8:::Q77.5:::N94.5:::K30:::G90.4:::G24.2:::N94.4:::Q78.5:::D50.1:::G24.4:::Q78.3:::G80.3:::R13.9:::Q75.4:::G90.48,Dyspnoe:::Dysphagie:::Dysurie:::Dyspareunie:::Dysthymia:::Dystonie:::Dysphonie:::Arzneimittelinduzierte Dystonie:::Dystelektase:::Dysplasia spondyloepiphysaria:::Sonstige Dystonie:::Diastrophische...
9,1.txt,Hust

### Combine two df

In [16]:
combined_df = pd.concat([result_pd_last, result_pd], axis=1)

In [17]:
#remove same column
df = combined_df.loc[:, ~combined_df.columns.duplicated()]

In [18]:
df.rename(columns={'text': 'chunk'}, inplace=True)

In [21]:
df.columns

Index(['file', 'chunk', 'icd10gm_code', 'icd10gm_description', 'all_codes',
       'resolutions', 'entity', 'begin', 'end'],
      dtype='object')

In [23]:
new_order = ["file","chunk","begin","end","entity","icd10gm_code","all_codes","resolutions","icd10gm_description"]

In [25]:
df = df.reindex(columns=new_order)
df

,file,chunk,begin,end,entity,icd10gm_code,all_codes,resolutions,icd10gm_description
0,1.txt,Bronchialkarzinom,17,33,MEDICAL_CONDITION,J47,J47:::J20:::J21:::J41.1:::Q33.4:::Q32.2:::Q32.3:::J41.0:::J45:::Z96.80:::J20.8:::J20.6:::J41:::D02.2:::J41.8:::J21.8:::J20.3:::J42:::J20.9:::J20.2:::D14.3:::J21.9:::J21.0:::T17.5:::J45.1,Bronchiektasen:::Akute Bronchitis:::Akute Bronchiolitis:::Schleimig-eitrige chronische Bronchitis:::Angeborene Bronchiektasie:::Angeborene Bronchomalazie:::Angeborene Bronchusstenose:::Einfache ch...,Bronchiektasen
1,1.txt,Lungenkrebs,50,60,MEDICAL_CONDITION,J81,J81:::A20.2:::A22.1:::I26:::Q22.0:::Z52.80:::B45.0:::I37.1:::Q33.3:::I37:::B40.1:::J98.1:::I37.0:::I37.8:::B46.0:::S27.32:::Q33.1:::C34:::B40.0:::C78.0:::Q26.3:::J44:::A42.0:::D02.2:::D14.3,Lungenödem:::Lungenpest:::Lungenmilzbrand:::Lungenembolie:::Pulmonalklappenatresie:::Lungenspender:::Kryptokokkose der Lunge:::Pulmonalklappeninsuffizienz:::Agenesie der Lunge:::Pulmonalklappenkra...,Lungenödem
2,1.txt,Hernia,73,78,MEDICAL_CONDITION,K42,K42:::K40:::K41:::K43:::K44:::O26.4:::K45:::I25.3:::K42.1:::B00.7:::K43.78:::L13.0:::B00.2:::Q99.1:::K44.1:::Q56.0:::K43.4:::K40.1:::K41.1:::B00.78:::K46:::B00.8:::K43.79:::B00.70:::K43.7,Hernia umbilicalis:::Hernia inguinalis:::Hernia femoralis:::Hernia ventralis:::Hernia diaphragmatica:::Herpes gestationis:::Sonstige abdominale Hernien:::Herz-(Wand-)Aneurysma:::Hernia umbilicalis...,Hernia umbilicalis
3,1.txt,Akne,91,94,MEDICAL_CONDITION,L70,L70:::L70.8:::L70.5:::L70.9:::L98.5:::A22.0:::H31:::L02.0:::C44:::D44.0:::M24.1:::O90.2:::I86.80:::M85.4:::A20.1:::T86.51:::C79.2:::C44.9:::L13:::C44.59:::H15:::M50:::L44:::H04.1:::Z52.3,"Akne:::Sonstige Akne:::Acné excoriée:::Akne, nicht näher bezeichnet:::Muzinose der Haut:::Hautmilzbrand:::Sonstige Affektionen der Aderhaut:::Hautabszess, Furunkel und Karbunkel im Gesicht:::Sonst...",Akne
4,1.txt,hochmalignes bronchogenes Karzinom,112,145,MEDICAL_CONDITION,T17.5,T17.5:::Z96.80:::J41.1:::Q33.1:::Q32.3:::D14.3:::J20.8:::Q24.3:::S27.31:::Q33.4:::J21.8:::R06.5:::A22.1:::J81:::Q32.4:::J21:::J20.9:::I42.8:::J39.1:::K83.5:::Q24.1:::C39.9:::M31.6:::J20:::J21.9,Fremdkörper im Bronchus:::Vorhandensein eines Bronchialstents:::Schleimig-eitrige chronische Bronchitis:::Akzessorischer Lungenlappen:::Angeborene Bronchusstenose:::Gutartige Neubildung: Bronchus ...,Fremdkörper im Bronchus
5,1.txt,Symptome,455,462,MEDICAL_CONDITION,R45,R45:::R39:::R45.8:::R39.8:::R46:::F68.0:::F34:::R46.8:::R29:::R68.1:::F45.4:::R63:::F34.9:::R63.8:::R29.8:::R44:::R44.8:::N64.5:::R68.8:::R09:::R09.8:::F32.3:::M05.08:::F38.8:::U63.1,"Symptome, die die Stimmung betreffen:::Sonstige Symptome, die das Harnsystem betreffen:::Sonstige Symptome, die die Stimmung betreffen:::Sonstige und nicht näher bezeichnete Symptome, die das Harn...","Symptome, die die Stimmung betreffen"
6,1.txt,Symptomen,472,480,MEDICAL_CONDITION,R39,R39:::R45:::R45.8:::R39.8:::R46:::F34:::F68.0:::R46.8:::R29:::R68.1:::R63:::F34.9:::F45.4:::R63.8:::R29.8:::R44:::R44.8:::R09:::N64.5:::R68.8:::R09.8:::F32.3:::F38.8:::M05.08:::U63.1,"Sonstige Symptome, die das Harnsystem betreffen:::Symptome, die die Stimmung betreffen:::Sonstige Symptome, die die Stimmung betreffen:::Sonstige und nicht näher bezeichnete Symptome, die das Harn...","Sonstige Symptome, die das Harnsystem betreffen"
7,1.txt,Thoraxschmerz,490,502,MEDICAL_CONDITION,C76.1,C76.1:::M96.81:::T33.2:::S20.2:::M41.35:::M41.34:::S22.5:::S20:::S29.8:::T34.2:::Z93.80:::M48.45:::S20.7:::S29:::Z43.80:::T04.1:::S21:::M41.3:::S20.8:::S29.7:::M49.54:::Q67.8:::S20.88:::S29.9:::M4...,Thorax:::Instabiler Thorax nach thoraxchirurgischem Eingriff:::Oberflächliche Erfrierung des Thorax:::Prellung des Thorax:::Thoraxbedingte Skoliose: Thorakolumbalbereich:::Thoraxbedingte Skoliose:...,Thorax
8,1.txt,Dyspnoe,505,511,MEDICAL_CONDITION,R06.0,R06.0:::R13:::R30.0:::N94.1:::F34.1:::G24:::R49.0:::G24.0:::J98.10:::Q77.7:::G24.8:::Q77.5:::N94.5:::K30:::G90.4:::G24.2:::N94.4:::Q78.5:::D50.1:::G24.4:::Q78.3:::G80.3::